# Activation Quantization

### Import Packages

In [ ]:
import tensorflow as tf
import json
from tensorflow.keras import layers, Sequential # type: ignore
from tensorflow.keras.applications.vgg16 import preprocess_input # type: ignore
from tensorflow.keras.layers import Conv2D, Dense # type: ignore
from tensorflow.keras.models import clone_model # type: ignore
# Local imports
from ml_project_util.path import path_definition
from ml_project_util.flatten_model import flatten_condtitional
from ml_project_util.model_evaluation import model_evaluation_precise
from ml_project_util.quantization_util import quant_activations

### Variable Paths, Names, Execution Environments

In [ ]:
# dict = path_definition(ds_rel_path='catsdogsconv/CatsDogs')
dict = path_definition()
BASE_PATH = dict['BASE_PATH']
PATH_DATASET = dict['PATH_DATASET']
PATH_TEST = dict['PATH_TEST']
PATH_RAWDATA = dict['PATH_RAWDATA']
PATH_JOINEDDATA = dict['PATH_JOINEDDATA']
PATH_SAVEDMODELS = dict['PATH_SAVEDMODELS']

### Load Float Model

In [ ]:
# Two first letters for dataset, next letter stands for parameter
# 3 numbers are for the number of epochs, and the last 4 digits are for validation loss
# e.g. CD4_P2_FT_003_val0.0336
model_name = ''
short_name = model_name[:-10]
parent_name = model_name[:3]
filepath = f'{PATH_SAVEDMODELS}/{parent_name}/{model_name}.keras'
model = tf.keras.models.load_model(filepath)
model = flatten_condtitional(model, model_name)
model.summary()

### Create New Model with Fake Quant Layers & Evaluate

In [ ]:
qa_model =  quant_activations(model, model_name, num_bits=8, mode_func='eval', mode='hw')